In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os
from rapidfuzz import process, fuzz

In [2]:
# OUTPUT: Processed_Data.csv

In [3]:
# Reading the cleaned data and product data
df_files = pd.read_csv("Clean_Data.csv")
df_products = pd.read_csv("../Format & Logistics Data/item_no.csv", encoding='latin1')

In [4]:

# Normalize for comparison
df_products['item_no'] = df_products['item_no'].astype(str).str.lower()
df_products['description'] = df_products['description'].astype(str).str.strip()
df_files['file_name'] = df_files['file_name'].astype(str).str.lower()

# Add empty columns for result
df_files['matched_item_no'] = None
df_files['matched_description'] = None
df_files['match_flag'] = 0

# Match item_no in file_name
for _, row in df_products.iterrows():
    item = row['item_no']
    desc = row['description']
    match_mask = df_files['file_name'].str.contains(item, na=False)

    # Only update unmatched rows to avoid overwriting
    unmatched = df_files['match_flag'] == 0
    update_mask = match_mask & unmatched

    df_files.loc[update_mask, 'matched_item_no'] = item
    df_files.loc[update_mask, 'matched_description'] = desc
    df_files.loc[update_mask, 'match_flag'] = 1


C:\Users\jpeinado\AppData\Local\Temp\ipykernel_34792\458968409.py:15: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  match_mask = df_files['file_name'].str.contains(item, na=False)


In [5]:
# Rename columns for clarity

columns_to_rename = {
    'parent_folder': 'prefix_name',
    'matched_item_no': 'item_no',
    'matched_description': 'description',
    'match_flag': 'item_no_match_flag'
}

df_files.rename(columns=columns_to_rename, inplace=True)
print(df_files.columns.tolist())


df_files = df_files[['item_no', 'file_name', 'description', 'prefix_name', 'pdf','dwg','sldprt', 'sldasm', 'ipt', 'iam','idw', 'file_path','main_folder', 'item_no_match_flag']]

df_files.head(2)


['prefix_name', 'file_name', 'file_path', 'pdf', 'iam', 'ipt', 'dwg', 'rfa', 'sldprt', 'sldasm', 'step', 'idw', 'main_folder', 'item_no', 'description', 'item_no_match_flag']


,item_no,file_name,description,prefix_name,pdf,dwg,sldprt,sldasm,ipt,iam,idw,file_path,main_folder,item_no_match_flag
0,plywood,4x9 plywood sheet,PLYWOOD,- 3D PARTS,0,1,0,0,0,0,0,H:\CACLA - Caledon ON\Product and Technical\Fa...,Fabrication,1
1,plywood,4x9 plywood sheet 2,PLYWOOD,- 3D PARTS,0,1,0,0,0,0,0,H:\CACLA - Caledon ON\Product and Technical\Fa...,Fabrication,1


In [6]:
df_files.to_csv("Processed_Data.csv", index=False)